In [1]:
from astropy.table import Table, Column, table, vstack
from pathlib import Path

import numpy as np
import astropy.units as u

from matplotlib import pyplot as plt

In [2]:
path_raw='../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
path_SED = '../Datos/Work/SEDs/'
path_CIGALE_phot = '../Datos/Work/CIGALE_InputPhot/'
path_CIGALE_out = '../Datos/Work/CIGALE_Output/'
path_FINAL = '../Datos/Final/'

In [3]:
date_sample = 'Apr_12_2022'
NeII_NeV = Table.read(path_raw+'Obj_Ne-IR_'+date_sample+'.vot',format='votable')

In [4]:
'''
to_rem = [] #Rows to be deleted or removed in out_1
for i in range(len(NeII_NeV)): 
    if (NeII_NeV['NED_Ne_IR_info'][i] != 'NeII|NeIII|NeV' and NeII_NeV['NED_Ne_IR_info'][i]!= 'NeII|NeV'):
        to_rem.append(i)
    
NeII_NeV.remove_rows(to_rem)
NeII_NeV.show_in_notebook()
'''

"\nto_rem = [] #Rows to be deleted or removed in out_1\nfor i in range(len(NeII_NeV)): \n    if (NeII_NeV['NED_Ne_IR_info'][i] != 'NeII|NeIII|NeV' and NeII_NeV['NED_Ne_IR_info'][i]!= 'NeII|NeV'):\n        to_rem.append(i)\n    \nNeII_NeV.remove_rows(to_rem)\nNeII_NeV.show_in_notebook()\n"

In [5]:
def Neon_info(NED_table, Freq):
    Ne_idx = [np.where(NED_table['Frequency'] == freq)[0] for freq in Freq]
    print(Ne_idx)
    print(len(Ne_idx))
    
    Ne_info = Table()
    for idx in range(len(Freq)):
        Ne_info = vstack([Ne_info,NED_table[Ne_idx[idx]]])

    Ne_inf = Ne_info['Observed_Passband','Frequency','Flux_Density','NED_Uncertainty','NED_Units','Refcode']
    return Ne_inf

In [6]:
passband = Table()

for id in range(len(NeII_NeV)):
    NED_phot = Table()
    obj = NeII_NeV['Main_Id'][id]
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    NED_phot.remove_rows(np.where(NED_phot['Flux_Density'].mask)[0])
    #NED_phot.remove_rows(np.where(NED_phot['NED_Units'] == 'Jy')[0])
    #NED_phot.remove_rows(np.where(NED_phot['NED_Uncertainty'] == '')[0])
    #NED_phot.remove_rows(np.where(NED_phot['NED_Uncertainty'] == '+/-...')[0])
    #NED_phot.remove_rows(np.where(NED_phot['Observed_Passband'] == '[Cl II] line (IRS)')[0])
    #NED_phot.remove_rows(np.where(NED_phot['Observed_Passband'] == '[Cl II] 14.4 (IRS)')[0])
    #NED_phot.remove_rows(np.where(NED_phot['Observed_Passband'] == '[Cl II] (IRS)')[0])
    #NED_phot.remove_rows(np.where(NED_phot['Observed_Passband'] == '[Cl II] 14.37 (IRS)')[0])
    
    NeII_NeV_Freq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0] #23400000000000.0
    NeII_NeV_inf = Neon_info(NED_phot, NeII_NeV_Freq)
    
    passband = vstack([passband,NeII_NeV_inf['Observed_Passband']])
    print(NeII_NeV_inf,'\n')

Galaxy:  3C 234.0
[array([], dtype=int64), array([147, 148]), array([144]), array([143]), array([141, 142])]
5
 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2010ApJ...709.1257T
[Ne III] 15.55 micrn 19300000000000.0 ...     Jy-Hz 2006ApJ...647..161O
  [Ne V] 14.32 (IRS) 20900000000000.0 ...     Jy-Hz 2010ApJ...709.1257T
 [Ne V] 14.3 microns 21000000000000.0 ...     Jy-Hz 2006ApJ...647..161O
  [Ne II] 12.8 (IRS) 23400000000000.0 ...     Jy-Hz 2009ApJ...697..182T
[Ne II] 12.81 micron 23400000000000.0 ...     Jy-Hz 2006ApJ...647..161O 

Galaxy:  M  84
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([201, 202]), array([], dtype=int64)]
5
 Observed_Passband     Frequency     ... NED_Units       Refcode      
                           Hz      

Galaxy:  NAME 3C 330G2
[array([21]), array([19, 20]), array([17]), array([14, 15, 16]), array([], dtype=int64)]
5
 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
[NeIII] 15.6 microns 19200000000000.0 ...     Jy-Hz 2007ApJ...655L..73G
[Ne III] 15.55 micrn 19300000000000.0 ...     Jy-Hz 2006ApJ...647..161O
 [Ne III] 15.5 (IRS) 19300000000000.0 ...     Jy-Hz 2007ApJ...660..117C
 [Ne V] 14.3 microns 20900000000000.0 ...     Jy-Hz 2007ApJ...655L..73G
 14.3 microns ISOCAM 21000000000000.0 ...        Jy 2004A&A...421..129S
 [Ne V] 14.3 microns 21000000000000.0 ...     Jy-Hz 2006ApJ...647..161O
   [Ne V] 14.3 (IRS) 21000000000000.0 ...     Jy-Hz 2007ApJ...660..117C 

Galaxy:  3C 265.0
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
5
Observed_Passband Frequ

 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2012ApJ...744....2A
 [Ne V] 14.3 Spitzer 21000000000000.0 ...     Jy-Hz 2011ApJ...730...28P
 [Ne II] 12.81 (IRS) 23400000000000.0 ...     Jy-Hz 2012ApJ...744....2A 

Galaxy:  [VV2000] J235225.0+244009
[array([], dtype=int64), array([32]), array([30]), array([], dtype=int64), array([29])]
5
 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2011ApJS..193...18W
  [Ne V] 14.32 (IRS) 20900000000000.0 ...     Jy-Hz 2011ApJS..193...18W
 [Ne II] 12.81 (IRS) 23400000000000.0 ...     Jy-Hz 2011ApJS..193...18W 

Galaxy:

 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
[NeIII] 15.6 microns 19200000000000.0 ...     Jy-Hz 2007ApJ...655L..73G
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2011ApJ...740...94D
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  [Ne V] 14.32 (IRS) 20900000000000.0 ...     Jy-Hz 2010ApJ...716.1151W
 [Ne V] 14.3 microns 20900000000000.0 ...     Jy-Hz 2007ApJ...655L..73G
  [Ne V] 14.32 (IRS) 20900000000000.0 ...     Jy-Hz 2011ApJ...740...94D
 [Ne II] 12.81 (IRS) 23400000000000.0 ...     Jy-Hz 2010ApJ...716.1151W
          NEII (VLT) 23400000000000.0 ...        Jy 2014MNRAS.439.1648A
 [Ne II] 12.81 (IRS) 23400000000000.0 ...     Jy-Hz 2011ApJ...740...94D 

Galaxy:  Mrk  463E
[array([], dtype=int64), array([28, 29, 30]), array([25, 26]), array([24]), array([21, 22, 23])]
5
 Observed_Passba

 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2011ApJ...740...94D
  [Ne V] 14.32 (IRS) 20900000000000.0 ...     Jy-Hz 2011ApJ...740...94D 

Galaxy:  LEDA   27453
[array([], dtype=int64), array([87]), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
5
 Observed_Passband     Frequency     ... NED_Units       Refcode      
                           Hz        ...                              
------------------- ---------------- ... --------- -------------------
[Ne III] 15.5 (IRS) 19300000000000.0 ...     Jy-Hz 2010ApJ...715..986S 

Galaxy:  NAME Cyg A
[array([134]), array([], dtype=int64), array([], dtype=int64), array([133]), array([129, 130])]
5
 Observed_Passband     Frequency     ... NED_Units       Refcode      
                           Hz

...

 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
  [Ne III] (Spitzer) 19300000000000.0 ...     Jy-Hz 2007ApJ...671..124D
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  [Ne V] 14.32 (IRS) 20900000000000.0 ...     Jy-Hz 2010ApJ...716.1151W
[Ne V] 14.32 Spitzer 20900000000000.0 ...     Jy-Hz 2007ApJ...671..124D
 [Ne II] 12.81 (IRS) 23400000000000.0 ...     Jy-Hz 2010ApJ...716.1151W 

Galaxy:  NGC  4725
[array([], dtype=int64), array([188]), array([], dtype=int64), array([], dtype=int64), array([187])]
5
 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
[NeIII] 15.6 Spitzer 19300000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
[Ne II] 12.8

[array([], dtype=int64), array([], dtype=int64), array([114]), array([], dtype=int64), array([113])]
5
 Observed_Passband     Frequency     ... NED_Units       Refcode      
                           Hz        ...                              
------------------- ---------------- ... --------- -------------------
 [Ne V] 14.32 (IRS) 20900000000000.0 ...     Jy-Hz 2011ApJ...740...94D
[Ne II] 12.81 (IRS) 23400000000000.0 ...     Jy-Hz 2011ApJ...740...94D 

Galaxy:  ESO 103-35
[array([139]), array([136, 137, 138]), array([132, 133, 134, 135]), array([], dtype=int64), array([127, 128, 129])]
5
 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
[NeIII] 15.6 microns 19200000000000.0 ...     Jy-Hz 2007ApJ...655L..73G
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2011ApJ...740...94D
[Ne III] 15.56 (IRS) 19300000000000.0 ... 

 Observed_Passband      Frequency     ... NED_Units       Refcode      
                            Hz        ...                              
-------------------- ---------------- ... --------- -------------------
      [Ne III] 15.56 19300000000000.0 ...     Jy-Hz 2011ApJS..195...17W
[Ne III] 15.56 (IRS) 19300000000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  [Ne III] (Spitzer) 19300000000000.0 ...     Jy-Hz 2007ApJ...671..124D
        [Ne V] 14.32 20900000000000.0 ...     Jy-Hz 2011ApJS..195...17W
  [Ne V] 14.32 (IRS) 20900000000000.0 ...     Jy-Hz 2010ApJ...709.1257T
       [Ne II] 12.81 23400000000000.0 ...     Jy-Hz 2011ApJS..195...17W
   [Ne II] (Spitzer) 23400000000000.0 ...     Jy-Hz 2007ApJ...671..124D
 [Ne II] 12.81 (IRS) 23400000000000.0 ...     Jy-Hz 2010ApJ...709.1257T 

Galaxy:  Mrk  463
[array([], dtype=int64), array([], dtype=int64), array([79]), array([], dtype=int64), array([], dtype=int64)]
5
Observed_Passband    Frequency     ... NED_Units       Refcode      
       

 Observed_Passband     Frequency     ... NED_Units       Refcode      
                           Hz        ...                              
------------------- ---------------- ... --------- -------------------
[Ne III] 15.6 (IRS) 19300000000000.0 ...     Jy-Hz 2009ApJ...693.1223S
 [Ne II] 12.8 (IRS) 23400000000000.0 ...     Jy-Hz 2009ApJ...693.1223S
 [Ne II] 12.8 (IRS) 23400000000000.0 ...     Jy-Hz 2004ApJS..154..184S 

Galaxy:  LBQS 1244+0238
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([124]), array([123])]
5
Observed_Passband    Frequency     ... NED_Units       Refcode      
                         Hz        ...                              
----------------- ---------------- ... --------- -------------------
     [Ne V] (IRS) 21000000000000.0 ...     Jy-Hz 2006ApJ...649...79S
    [Ne II] (IRS) 23400000000000.0 ...     Jy-Hz 2006ApJ...649...79S 

Galaxy:  NGC  6860
[array([], dtype=int64), array([ 98,  99, 100]), array([94, 95, 96]), array([],

 Observed_Passband     Frequency     ... NED_Units       Refcode      
                           Hz        ...                              
------------------- ---------------- ... --------- -------------------
[Ne V] 14.3 Spitzer 20900000000000.0 ...     Jy-Hz 2011MNRAS.411.1231G 

Galaxy:  2MASX J00215355-7910077
[array([], dtype=int64), array([39]), array([38]), array([], dtype=int64), array([37])]
5
Observed_Passband    Frequency     ... NED_Units       Refcode      
                         Hz        ...                              
----------------- ---------------- ... --------- -------------------
   [Ne III] 15.56 19300000000000.0 ...     Jy-Hz 2011ApJS..195...17W
     [Ne V] 14.32 20900000000000.0 ...     Jy-Hz 2011ApJS..195...17W
    [Ne II] 12.81 23400000000000.0 ...     Jy-Hz 2011ApJS..195...17W 

Galaxy:  NGC  3783
[array([243]), array([240, 241, 242]), array([235, 236, 237, 238, 239]), array([], dtype=int64), array([232, 233, 234])]
5
 Observed_Passband      Frequency

In [7]:
passbands, counts = np.unique(passband, return_counts=True)
passbands

array([('12.8 microns',), ('12.8 microns (ISO)',),
       ('12.81 microns (VLT)',), ('14.3 microns (ISO)',),
       ('14.3 microns ISOCAM',), ('15 microns (AKARI)',),
       ('15 microns (ISOCAM)',), ('NEII (VLT)',), ('Ne II (ESO)',),
       ('Ne II (VLT)',), ('Ne II (VLT/VISIR)',), ('NeII (Subaru)',),
       ('[Cl II] (IRS)',), ('[Cl II] 14.37 (IRS)',),
       ('[Cl II] 14.4 (IRS)',), ('[Cl II] line (IRS)',),
       ('[Ne III] (IRS)',), ('[Ne III] (Spitzer)',),
       ('[Ne III] 12.56 (IRS)',), ('[Ne III] 15 Spitzer',),
       ('[Ne III] 15.5 (IRS)',), ('[Ne III] 15.55 (IRS)',),
       ('[Ne III] 15.55 micrn',), ('[Ne III] 15.56',),
       ('[Ne III] 15.56 (IRS)',), ('[Ne III] 15.6 (IRS)',),
       ('[Ne III] line (IRS)',), ('[Ne II] (GeminiS)',),
       ('[Ne II] (IRS)',), ('[Ne II] (ISO)',), ('[Ne II] (Spitzer)',),
       ('[Ne II] (TIMMI2)',), ('[Ne II] 12.8 (IRS)',),
       ('[Ne II] 12.8 Spitzer',), ('[Ne II] 12.81',),
       ('[Ne II] 12.81 (IRS)',), ('[Ne II] 12.81 micron',),
 